# ALEX17 Diurnal Cycles Benchmark: A Large Domain In Complex Terrain
## Introduction
This is the model evaluation notebook of the ALEX17 benchmark. A guide for participants can be found [here](https://thewindvaneblog.com/alex17-diurnal-cycles-benchmark-a-large-domain-in-complex-terrain-b5029e94485).

## Load libraries

In [ ]:
%matplotlib inline
# Python packages
import os, datetime, sys
import numpy as np
import pandas as pd
import matplotlib as plt
from pathlib import Path
import warnings
# local functions
from lib.common_functions import ensure_dir
from lib.benchmark.alex17_functions import read_sim, read_obs
from lib.benchmark.alex17_plots import *
from lib.benchmark.alex17_qoi import *

#from lib.alex17_functions import *  
from IPython.display import Markdown, display
import ipywidgets as widgets

# jupyter lines
%load_ext autoreload
%autoreload 2

In [ ]:
%%javascript 
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false
}

## Participants

In [ ]:
# Set the csv filename with simulation's id and descriptions.
sim_list_file = 'alex17_sims'

# read table with models and simulation's description
sims = pd.read_csv(Path('./outputs')/(sim_list_file+'.csv'), comment='#')
sims

## Input Data
### Input options

In [ ]:
# Choose whether to save tables and figures as png files
saveFigures = False
saveTables  = False

# set the desired dip for image quality (if saveFigures is True)
dpi_fig = 100

# set paths of output files and input observations
observations_path   = Path('./observations')
output_path_figures = Path('./output_figures') / sim_list_file

# make sure directories exist. Created them otherwise
ensure_dir(output_path_figures) if saveFigures else ''
ensure_dir(observations_path) if saveTables else ''

### Instrument data, layout and Elevation Map

In [ ]:
# Read file of mast names and position
masts = pd.read_csv(Path("./inputs/masts.csv"))
masts = masts.drop(masts[masts['Name']=='WLS70'].index)

# Read Z-transect points' location
Ztransect = pd.read_csv('./inputs/Ztransect.csv') # read {x,y,x} coordinates of Z-transect

# digital elevation map file
tif_topofile = Path('./inputs/DTM_Alaiz_2m.tif')

# plot overall site characteristics
f, a = plot_topography(tif_topofile, Ztransect, masts)
saveFigureFunction(saveFigures, f, output_path_figures / ('overview.png'), dpi_fig)

### Mesoscale Input Data
Mesoscale simulations have been produced adopting most of the WRF settings that were used in the production run of the New European Wind Atlas (NEWA). Three one-way nested domains of 27, 9 and 3 km resolution are configured centred at the wind farm centroid. The vertical grid has 61 terrain-following (sigma) levels, with 10 levels covering the first 200 meters, more specifically at: 6, 22, 40, 56, 73, 90, 113, 140, 179 and 205 meters. High resolution topography (SRTM 90m) and updated land use categories (Corine Land Cover 2018), together with the Noah land-surface model are used to define the boundary conditions at the surface. The physical parameterizations are: Mellor–Yamada–Nakanishi Niino 2.5-level planetary boundary-layer scheme (MYNN), WRF Single-Moment 5-class microphysics scheme, the Rapid Radiative Transfer Model for GCMs shortwave and longwave radiation schemes and the Kain-Fritsch cumulus scheme in the outermost domains 1 and 2. The simulation is driven by input data from ERA-5 in blocks of 5 days with additional spin-up time of 24 hours.   

From the 3-km domain we extract output data that will be used as reference in validation plots. This constitute the baseline simulation (*alex17_00a*) that we try to improve with microscale modeling. 

## Validation Data
Three sets of measurements will be used for validation of the submitted results. The meteorological mast measurements are available for the modelers to check and calibrate their results, whereas the lidar measurements will be used for validation after submission (blind test). 
1. Vertical profiles at *mast* positions.
2. Wind vector at 125m a.g.l. along the *Z-transect*.
3. Vertical RHI plane across the valley along the *B* segment of the *Z-transect*. The Plane connects WS3 [617846.796; 4732496.246] km and WS5 [617305.590; 4729848.751] km.

## Mast locations

In [ ]:
# Load mast data contained in the observations netcdf file
masts_obs = read_obs(Path('./observations/masts_obs.nc'))


## Simulation Data
We load the simulations defined in *sims* table. Three sets of output data are collected from each simulation: 

1. *simID_masts.nc*: Vertical profiles extending 1km from the surface at the met mast locations (*inputs/masts.csv* coordinates);
2. *simID_Ztransect.nc*: Vertical planes along the Z-transect extending 1 km in the vertical (*inputs/Ztransect.csv* coordinates);
3. *simID_box.nc*: Horizontal planes at 125 m and 40 m a.g.l., with horizontal resolution of 100 m, within the “box” [612–622; 4726–4736] km.

Output data is provided based on 10-min averages.

In [ ]:
# Load dataset at masts positions from simulations files listed in the alex17_sims.csv
masts_sim = {x:read_sim(Path('./outputs/' + x +'_masts.nc')) for x in sims['ID'].values }


# Results at masts locations
## Vertical profiles

In [ ]:
# set events when different atm stability conditions ocurred and set a color to plot frames in the time series
events = {'all'        :[datetime.datetime(2018,9,30,0,0,0),  datetime.datetime(2018,10,4,0,0,0),  'white'      ],
          'neutral'    :[datetime.datetime(2018,10,1,18,0,0), datetime.datetime(2018,10,1,20,0,0), 'silver'     ],
          'stable'     :[datetime.datetime(2018,10,2,3,0,0),  datetime.datetime(2018,10,2,5,0,0),  'lightblue'  ],
          'unstable'   :[datetime.datetime(2018,10,2,12,0,0), datetime.datetime(2018,10,2,14,0,0), 'salmon'     ],
          'very stable':[datetime.datetime(2018,10,3,4,0,0) , datetime.datetime(2018,10,3,6,0,0),  'deepskyblue']}


In [ ]:
# List the mast's id you wish to plot
masts2plot = ['MP5', 'M1', 'M2','M3','M5', 'M6', 'M7']

#-------------------- Vertical profiles-------------------------------
# select the events you want to plot
events2plot = ['all','neutral','stable','unstable','very stable']
events2plot = ['all']

# list the variables and their labels
vars2plot    = [
               ['wind_speed',           r'wind speed [$m s^{-1}$]'],
               ['wind_direction',        r'wind direction [deg]'],
               ['turbulence_intensity',  r'turbulence intensity'],
               # ['friction_velocity',     r'$u^*$'],
               # ['heat_flux',             r'heat flux [$m K s^{-1}$]'],
               ]

for ev in events2plot:
    for ivar in vars2plot:
        print("--plotting: "+ivar[0]+' ---------- cases: ' +ev)
        f, ax = masts_sims_vs_obs_profiles_plot(events[ev], ivar, masts_obs, masts_sim, masts2plot)
        ax[-1][-1].set_title('Episode: ' + ev )
        saveFigureFunction(saveFigures, f, output_path_figures / ('vertical_profiles_'+ivar[0]+'_'+ev), dpi_fig)

## Time series type of plots

In [ ]:
# set the height to plot the time series
h = 80

# List the mast's id you wish to plot
masts2plot = ['MP5', 'M1', 'M2','M3','M5', 'M6', 'M7']
masts2plot = ['MP5']

# set the variables to plot
vars2plot_ts = [
               ['wind_speed',              r'wind speed [$m s^{-1}$]'],
               ['wind_direction',          r'wind direction [deg]'],
               ['turbulence_intensity',    r'turbulence intensity'],
               ['wind_shear', r'wind shear $\alpha(80/40)$'],
               # ['stability',  r'stability $z/L$ ($z$=10m)']
               ]

# Set x-limits of the time series plot
datefrom, dateto = datetime.datetime(2018,9,30,0,0,0), datetime.datetime(2018,10,4,0,0,0)

for mast in masts2plot:
    f, ax = mast_sims_vs_obs_timeseries_plot(mast, h, vars2plot_ts, masts_obs, masts_sim, datefrom, dateto, events)
    saveFigureFunction(saveFigures, f, output_path_figures / ('time_'+mast), dpi_fig)



## Observation time series comparison

In [ ]:
# set the height to plot the time series
h = 80.
# List the mast's id you wish to plot
mast = ['MP5','M7', 'M5']

f, ax = compare_masts_timeseries_plot(mast, h, vars2plot_ts, masts_obs, datefrom, dateto, events)
saveFigureFunction(saveFigures, f, output_path_figures / ('mast_compare_'+mast[0]+'-'+mast[1]), dpi_fig)


### Quantities of interest

In [ ]:
# set the height to get the statistics
h = 80.

# get overall statistics (mean and std) of the observations at masts
masts_obs_quois = table_statistics_mast(events, masts_obs, h)
masts_obs_quois['MP5']


In [ ]:
# get the overall data availability of each mast and each varaible
obs_avai = table_availability_mast(masts_obs, output_path_figures)
obs_avai.T

## Bin-averages

In [ ]:
# set the mast and heigth to use as reference
mastID = 'M7'
h = 80

WDbins = np.arange(-5.,360.+5.,10)  # wind direction bins (12 sectors)
WDbins_label = np.array([str(k+1) for k in range(36)])

zLbins = np.array([-0.2,-0.02, 0.02, 0.2])      # 3 stability bins
zLbins_label = np.array(['u','n','s'])

bins_label = [WDbins_label, zLbins_label]

WDref = masts_obs.wind_direction.sel(id = mastID, height = h)
Sref = masts_obs.wind_speed.sel(id = mastID, height = h)
zLref = masts_obs.stability.sel(id = mastID, height = 10)

N_WDzL, binmap = WDzL_bins(WDref,zLref,Sref,'count',[WDbins, zLbins],[WDbins_label, zLbins_label],plot = False)
N_WDzL.T

In [ ]:
# main statistics from the observations at all masts
for h in masts_obs.height:
    height = 80.
    obs_table = table_statistics_mast(events, masts_obs, h)
    save_dftable(saveTables, obs_table, output_path_figures, 'Observation_statistics_'+str(int(h))+'m')

qoi = ['wind_speed', 'wind_direction', 'turbulence_intensity','wind_shear']
bias_table, mae_table = table_overall_errors(events, masts_obs, masts_sim, qoi, output_path_figures)
